# ทดสอบ Backend API - คู่มือสำหรับมือใหม่

**สำหรับ Person 4: ผู้ดูแล LLM Backend**

## เกี่ยวกับ Notebook นี้

Notebook นี้ใช้สำหรับทดสอบ Backend API ที่แปลงคำภาษามือเป็นประโยคไทย:
- ตรวจสอบการตั้งค่า API Keys
- ทดสอบการเชื่อมต่อ LLM (Gemini/OpenAI/Typhoon)
- สร้างประโยคไทยจากคำภาษามือ
- ประเมินคุณภาพของประโยคที่ได้

## ขั้นตอนการทำงาน

1. ตรวจสอบและตั้งค่า Environment
2. ติดตั้ง Libraries ที่จำเป็น
3. สร้างฟังก์ชัน Backend API
4. ทดสอบกับคำตัวอย่าง
5. ทดสอบหลายกรณี
6. ประเมินคุณภาพผลลัพธ์
7. เตรียมสำหรับการใช้งานจริง

## วิธีใช้งาน

o รันทุกเซลล์จากข้างบนตามลำดับ
o ใส่ API Key ในไฟล์ .env ตามที่แนะนำ
o ตรวจสอบผลลัพธ์ในแต่ละขั้นตอน

In [ ]:
# ขั้นตอนที่ 1: ตั้งค่า Environment และ API Keys

import os
from dotenv import load_dotenv

print("กำลังตั้งค่า Environment...")

# โหลดไฟล์ .env (สร้างถ้ายังไม่มี)
env_file = ".env"
if not os.path.exists(env_file):
    print("ไม่พบไฟล์ .env - กำลังสร้างไฟล์ตัวอย่าง...")
    
    with open(env_file, 'w', encoding='utf-8') as f:
        f.write("""# Thai-HandMate Backend Configuration
# คู่มือการขอ API Key: ดู README.md

# Google Gemini API (แนะนำ - ฟรี)
GEMINI_API_KEY=your_gemini_api_key_here
GEMINI_API_BASE=https://generativelanguage.googleapis.com/v1beta/models/gemini-pro:generateContent

# OpenAI API (ทางเลือก - ต้องจ่าย)
OPENAI_API_KEY=your_openai_api_key_here
OPENAI_API_BASE=https://api.openai.com/v1/chat/completions

# ตั้งค่าโมเดล
LLM_PROVIDER=gemini
""")
    print("สร้างไฟล์ .env ตัวอย่างแล้ว - กรุณาใส่ API Key ของคุณ")

# โหลดค่าตั้งค่า
load_dotenv()

# ตรวจสอบการตั้งค่า
gemini_key = os.getenv('GEMINI_API_KEY', '')
openai_key = os.getenv('OPENAI_API_KEY', '')
provider = os.getenv('LLM_PROVIDER', 'gemini')
typhoon_key = os.getenv('TYPHOON_API_KEY', '')

print("\nสถานะการตั้งค่า:")
print(f"- LLM Provider: {provider}")
print(f"- Gemini API Key: {'✅ มี' if gemini_key and gemini_key != 'your_gemini_api_key_here' else '❌ ไม่มี'}")
print(f"- OpenAI API Key: {'✅ มี' if openai_key and openai_key != 'your_openai_api_key_here' else '❌ ไม่มี'}")

if provider == 'gemini' and (not gemini_key or gemini_key == 'your_gemini_api_key_here'):
    print("\nกรุณาใส่ Gemini API Key ในไฟล์ .env")
    print("วิธีขอ API Key: ดู backend/README.md")
elif provider == 'openai' and (not openai_key or openai_key == 'your_openai_api_key_here'):
    print("\nกรุณาใส่ OpenAI API Key ในไฟล์ .env")
else:
    print(f"\nพร้อมใช้งาน {provider.upper()} API!")

In [ ]:
# ขั้นตอนที่ 2: ติดตั้ง Libraries

print("+ กำลังติดตั้ง libraries สำหรับ Backend API...")

# ติดตั้ง libraries ที่จำเป็น
import sys
!{sys.executable} -m pip install python-dotenv httpx asyncio

print("+ กำลัง import libraries...")

# Import libraries ที่จำเป็น
import os
import json
import asyncio
from dotenv import load_dotenv
from typing import List

try:
    import httpx
    print("  + httpx: ✅ พร้อมใช้งาน")
except ImportError:
    print("  - httpx: ❌ ติดตั้งไม่สำเร็จ")

print("+ Libraries พร้อมใช้งาน!")
print("="*50)

In [ ]:
# ขั้นตอนที่ 3: สร้างฟังก์ชัน Fallback (สำรอง)

print("+ กำลังสร้างฟังก์ชันสำรอง...")

def generate_fallback_sentences(words_text: str) -> List[str]:
    """
    สร้างประโยคแบบ fallback เมื่อ API ไม่พร้อมใช้งาน
    ใช้วิธีการง่ายๆ ในการสร้างประโยค
    """
    
    # ประโยคพื้นฐาน (ใช้คำเดิม)
    basic = words_text
    
    # เพิ่มคำสุภาพ
    polite = f"{words_text} ครับ/ค่ะ"
    
    # เพิ่มบริบทการสื่อสার
    contextual = f"ฉันต้องการสื่อว่า {words_text}"
    
    # รูปแบบคำถาม (สำหรับบางกรณี)
    question = f"คุณ{words_text}ไหม?"
    
    return [basic, polite, contextual]

# ทดสอบฟังก์ชัน fallback
test_words = "สวัสดี ขอบคุณ"
fallback_result = generate_fallback_sentences(test_words)

print("+ ทดสอบฟังก์ชัน Fallback:")
print(f"  คำทดสอบ: {test_words}")
for i, sentence in enumerate(fallback_result, 1):
    print(f"  {i}. {sentence}")

print("+ Fallback ฟังก์ชันพร้อมใช้งาน!")
print("="*50)

In [ ]:
# ขั้นตอนที่ 4: สร้างฟังก์ชัน LLM API

print("+ กำลังสร้างฟังก์ชัน LLM API...")

async def generate_with_gemini(words: List[str]) -> List[str]:
    """
    สร้างประโยคด้วย Google Gemini API
    """
    import httpx
    
    words_text = " ".join(words)
    
    prompt = f"""คุณเป็นผู้ช่วยที่เชี่ยวชาญในการสร้างประโยคภาษาไทยจากคำศัพท์ภาษามือ

จากคำเหล่านี้: {words_text}

กรุณาสร้างประโยคไทย 2-3 ประโยค ที่:
1. เป็นธรรมชาติและใช้ในชีวิตประจำวันได้
2. ใช้คำที่ให้มาทั้งหมดหรือส่วนใหญ่
3. สื่อความหมายได้ชัดเจน

ตอบเป็นรายการประโยค เรียงตามความเหมาะสม"""

    gemini_key = os.getenv('GEMINI_API_KEY', '')
    url = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-pro:generateContent?key={gemini_key}"
    
    payload = {
        "contents": [
            {
                "parts": [
                    {"text": prompt}
                ]
            }
        ]
    }
    
    async with httpx.AsyncClient() as client:
        response = await client.post(url, json=payload, timeout=30.0)
        
        if response.status_code != 200:
            raise Exception(f"Gemini API Error: {response.status_code}")
        
        data = response.json()
        content = data['candidates'][0]['content']['parts'][0]['text']
        
        # แปลงข้อความเป็นรายการประโยค
        sentences = []
        for line in content.strip().split('\n'):
            line = line.strip()
            if line and not line.startswith('#'):
                # ลบหมายเลข หรือ bullet points
                clean_line = line.lstrip('123456789.- ')
                if clean_line:
                    sentences.append(clean_line)
        
        return sentences[:3]  # เอาแค่ 3 ประโยคแรก

async def generate_with_openai(words: List[str]) -> List[str]:
    """
    สร้างประโยคด้วย OpenAI API (ทางเลือก)
    """
    import httpx
    
    words_text = " ".join(words)
    
    system_prompt = """คุณเป็นผู้ช่วยที่เชี่ยวชาญในการสร้างประโยคภาษาไทยจากคำศัพท์ภาษามือ ให้สร้างประโยคที่เป็นธรรมชาติและสื่อความหมายได้ชัดเจน"""

    user_prompt = f"""จากคำเหล่านี้: {words_text}
กรุณาสร้างประโยคไทย 2-3 ประโยค ที่เป็นธรรมชาติและใช้ในชีวิตประจำวันได้"""

    payload = {
        "model": "gpt-3.5-turbo",
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        "max_tokens": 300,
        "temperature": 0.7
    }

    headers = {
        "Authorization": f"Bearer {os.getenv('OPENAI_API_KEY', '')}",
        "Content-Type": "application/json"
    }

    async with httpx.AsyncClient() as client:
        response = await client.post(
            "https://api.openai.com/v1/chat/completions",
            json=payload,
            headers=headers,
            timeout=30.0
        )
        
        if response.status_code != 200:
            raise Exception(f"OpenAI API Error: {response.status_code}")
        
        data = response.json()
        content = data['choices'][0]['message']['content']
        
        # แปลงเป็นรายการประโยค
        sentences = []
        for line in content.strip().split('\n'):
            line = line.strip()
            if line and not line.startswith('#'):
                clean_line = line.lstrip('123456789.- ')
                if clean_line:
                    sentences.append(clean_line)
        
        return sentences[:3]

print("+ LLM API ฟังก์ชันพร้อมใช้งาน!")
print("  - generate_with_gemini() สำหรับ Google Gemini")
print("  - generate_with_openai() สำหรับ OpenAI")
print("="*50)

In [ ]:
# ขั้นตอนที่ 5: สร้างฟังก์ชันหลัก Backend API

print("+ กำลังสร้างฟังก์ชันหลัก generate_sentences()...")

async def generate_sentences(words: List[str]) -> dict:
    """
    ฟังก์ชันหลักสำหรับสร้างประโยคไทย - เหมือนกับใน app.py
    """
    
    if not words:
        return {"error": "ต้องระบุคำอย่างน้อย 1 คำ", "success": False}
    
    words_text = " ".join(words)
    print(f"  o กำลังสร้างประโยคจากคำ: {words_text}")
    
    # ตรวจสอบ API Keys และเลือก Provider
    gemini_key = os.getenv('GEMINI_API_KEY', '')
    openai_key = os.getenv('OPENAI_API_KEY', '')
    provider = os.getenv('LLM_PROVIDER', 'gemini')
    
    # ลองใช้ API ตามลำดับความสำคัญ
    try:
        if provider == 'gemini' and gemini_key and gemini_key != 'your_gemini_api_key_here':
            print("  + กำลังใช้ Google Gemini API...")
            sentences = await generate_with_gemini(words)
            return {
                "sentences": sentences,
                "provider": "gemini",
                "success": True,
                "words_used": words
            }
            
        elif provider == 'openai' and openai_key and openai_key != 'your_openai_api_key_here':
            print("  + กำลังใช้ OpenAI API...")
            sentences = await generate_with_openai(words)
            return {
                "sentences": sentences,
                "provider": "openai", 
                "success": True,
                "words_used": words
            }
            
        else:
            print(f"  - API {provider} ไม่พร้อมใช้งาน")
            
    except Exception as e:
        print(f"  - เกิดข้อผิดพลาดกับ {provider} API: {e}")
        print("  o กำลังใช้ระบบสำรอง...")
    
    # Fallback: สร้างประโยคแบบง่าย
    print("  + ใช้ระบบสำรอง (Fallback)...")
    fallback_sentences = generate_fallback_sentences(words_text)
    return {
        "sentences": fallback_sentences,
        "provider": "fallback",
        "success": True,
        "words_used": words
    }

print("+ ฟังก์ชันหลัก Backend API พร้อมใช้งาน!")
print("  - รองรับ Google Gemini API")  
print("  - รองรับ OpenAI API")
print("  - มีระบบสำรองเมื่อ API ไม่พร้อม")
print("="*50)

In [ ]:
# ขั้นตอนที่ 6: ทดสอบกับคำตัวอย่าง

print("+ เริ่มทดสอบ Backend API...")

# ตัวอย่างคำจากภาษามือไทย
test_words = ["ผม", "รัก", "คุณ"]

print(f"  o ทดสอบกับคำ: {test_words}")
print("  o กำลังสร้างประโยค...")

# เรียกใช้ฟังก์ชันหลัก
result = await generate_sentences(test_words)

print("\n+ ผลลัพธ์การทดสอบ:")
print(f"  - Provider: {result.get('provider', 'unknown')}")
print(f"  - สถานะ: {'สำเร็จ' if result.get('success') else 'ล้มเหลว'}")

if result.get('success'):
    print(f"  - คำที่ใช้: {' '.join(result.get('words_used', []))}")
    print("\n  ประโยคที่ได้:")
    for i, sentence in enumerate(result.get('sentences', []), 1):
        print(f"    {i}. {sentence}")
else:
    print(f"  - ข้อผิดพลาด: {result.get('error', 'ไม่ทราบสาเหตุ')}")

print("="*50)

In [ ]:
# ขั้นตอนที่ 7: ทดสอบหลายกรณี

print("+ ทดสอบด้วยคำหลายแบบ...")

# กรณีทดสอบต่างๆ (จากคำภาษามือจริง)
test_cases = [
    {
        "name": "คำเดียว - การทักทาย",
        "words": ["สวัสดี"]
    },
    {
        "name": "สองคำ - ขอบคุณ + ตกลง",
        "words": ["ขอบคุณ", "โอเค"]
    },
    {
        "name": "สามคำ - ประโยคพื้นฐาน",
        "words": ["ผม", "ไป", "บ้าน"]
    },
    {
        "name": "สี่คำ - ประโยคซับซ้อน",
        "words": ["คุณ", "สวัสดี", "ขอโทษ", "มา"]
    },
    {
        "name": "คำอารมณ์",
        "words": ["ดีใจ", "รัก", "คิดถึง"]
    }
]

print(f"\n  o จะทดสอบทั้งหมด {len(test_cases)} กรณี\n")

successful_tests = 0
failed_tests = 0

for i, test_case in enumerate(test_cases, 1):
    print(f"กรณีที่ {i}: {test_case['name']}")
    print(f"  คำทดสอบ: {test_case['words']}")
    
    try:
        result = await generate_sentences(test_case['words'])
        
        if result.get('success'):
            successful_tests += 1
            print(f"  + ผล: สำเร็จ ({result.get('provider')})")
            
            for j, sentence in enumerate(result.get('sentences', []), 1):
                print(f"    {j}. {sentence}")
        else:
            failed_tests += 1
            print(f"  - ผล: ล้มเหลว ({result.get('error', '')})")
            
    except Exception as e:
        failed_tests += 1
        print(f"  - ข้อผิดพลาด: {e}")
    
    print()

# สรุปผลการทดสอบ
print("+ สรุปผลการทดสอบ:")
print(f"  - ทดสอบทั้งหมด: {len(test_cases)} กรณี")
print(f"  - สำเร็จ: {successful_tests} กรณี")
print(f"  - ล้มเหลว: {failed_tests} กรณี")
print(f"  - อัตราความสำเร็จ: {(successful_tests/len(test_cases)*100):.1f}%")

if successful_tests >= len(test_cases) * 0.8:
    print("  + ยอดเยี่ยม! ระบบทำงานได้ดี")
elif successful_tests >= len(test_cases) * 0.6:
    print("  o ดี ใช้งานได้")
else:
    print("  - ควรตรวจสอบการตั้งค่า API")

print("="*50)

In [ ]:
# ขั้นตอนที่ 8: ประเมินคุณภาพและประสิทธิภาพ

print("+ กำลังประเมินคุณภาพของ Backend API...")

import time

async def test_api_performance():
    """
    ทดสอบประสิทธิภาพของ API
    """
    print("  o ทดสอบความเร็วในการตอบสนอง...")
    
    test_words = ["ผม", "รัก", "คุณ"]
    times = []
    
    # ทดสอบ 5 ครั้ง
    for i in range(5):
        start_time = time.time()
        result = await generate_sentences(test_words)
        end_time = time.time()
        
        response_time = end_time - start_time
        times.append(response_time)
        
        print(f"    ครั้งที่ {i+1}: {response_time:.2f} วินาที")
    
    # คำนวณสถิติ
    avg_time = sum(times) / len(times)
    min_time = min(times)
    max_time = max(times)
    
    print(f"\n  สถิติการตอบสนอง:")
    print(f"    + เวลาเฉลี่ย: {avg_time:.2f} วินาที")
    print(f"    + เวลาเร็วสุด: {min_time:.2f} วินาที")
    print(f"    + เวลาช้าสุด: {max_time:.2f} วินาที")
    
    # ประเมินประสิทธิภาพ
    if avg_time <= 2:
        performance = "ดีเยี่ยม"
        status = "+"
    elif avg_time <= 5:
        performance = "ดี"
        status = "o"
    elif avg_time <= 10:
        performance = "พอใช้"
        status = "o"
    else:
        performance = "ช้า"
        status = "-"
        
    print(f"  {status} ประเมินประสิทธิภาพ: {performance}")
    
    return {
        'avg_time': avg_time,
        'performance': performance,
        'times': times
    }

# รันการทดสอบประสิทธิภาพ
perf_result = await test_api_performance()

print("\n+ ตรวจสอบรูปแบบ JSON Response:")

# ทดสอบรูปแบบการตอบกลับ
test_result = await generate_sentences(["ผม", "รัก", "คุณ"])
print("  o ตัวอย่าง JSON Response:")
print(json.dumps(test_result, ensure_ascii=False, indent=2))

print("\n+ ข้อแนะนำสำหรับการปรับปรุง:")
if perf_result['performance'] in ['ดีเยี่ยม', 'ดี']:
    print("  + API ทำงานได้ดี พร้อมใช้งาน Production")
else:
    print("  o ควรปรับปรุงประสิทธิภาพ:")
    print("    - ตรวจสอบการเชื่อมต่อ Internet")
    print("    - เปลี่ยน API Provider")
    print("    - ลดความซับซ้อนของ Prompt")

print("="*50)

In [ ]:
# ขั้นตอนที่ 9: เตรียมใช้งานจริงและสรุป

print("+ เตรียมความพร้อมสำหรับการใช้งานจริง...")

# ตรวจสอบความพร้อม
def check_production_readiness():
    """
    ตรวจสอบความพร้อมสำหรับ Production
    """
    
    checks = []
    
    # ตรวจสอบ API Keys
    gemini_key = os.getenv('GEMINI_API_KEY', '')
    openai_key = os.getenv('OPENAI_API_KEY', '')
    
    if gemini_key and gemini_key != 'your_gemini_api_key_here':
        checks.append({"name": "Gemini API Key", "status": True})
    else:
        checks.append({"name": "Gemini API Key", "status": False})
    
    if openai_key and openai_key != 'your_openai_api_key_here':
        checks.append({"name": "OpenAI API Key", "status": True})
    else:
        checks.append({"name": "OpenAI API Key", "status": False})
        
    # ตรวจสอบ Libraries
    try:
        import httpx
        checks.append({"name": "httpx Library", "status": True})
    except ImportError:
        checks.append({"name": "httpx Library", "status": False})
        
    try:
        from dotenv import load_dotenv
        checks.append({"name": "python-dotenv", "status": True})
    except ImportError:
        checks.append({"name": "python-dotenv", "status": False})
    
    # ตรวจสอบไฟล์ app.py
    app_exists = os.path.exists('app.py')
    checks.append({"name": "app.py ไฟล์", "status": app_exists})
    
    # ตรวจสอบไฟล์ .env
    env_exists = os.path.exists('.env')
    checks.append({"name": ".env ไฟล์", "status": env_exists})
    
    return checks

# รันการตรวจสอบ
readiness_checks = check_production_readiness()

print("  o ผลการตรวจสอบความพร้อม:")
passed_checks = 0

for check in readiness_checks:
    status_icon = "✅" if check["status"] else "❌"
    print(f"    {status_icon} {check['name']}")
    if check["status"]:
        passed_checks += 1

readiness_score = (passed_checks / len(readiness_checks)) * 100

print(f"\n  + คะแนนความพร้อม: {readiness_score:.1f}%")

if readiness_score >= 80:
    print("  + ยอดเยี่ยม! พร้อมใช้งาน Production")
    next_steps = [
        "รัน Backend Server ด้วย: python app.py",
        "ทดสอบ API ด้วย Frontend",
        "Deploy ขึ้น Server จริง"
    ]
elif readiness_score >= 60:
    print("  o ใกล้พร้อมแล้ว แต่ควรแก้ไขข้อบกพร่อง")
    next_steps = [
        "แก้ไขข้อมูลที่ขาดหาย",
        "ทดสอบอีกครั้ง",
        "รัน Backend Server"
    ]
else:
    print("  - ยังไม่พร้อม ต้องแก้ไขหลายอย่าง")
    next_steps = [
        "ติดตั้ง Libraries ที่ขาดหาย",
        "ตั้งค่า API Keys",
        "ทดสอบระบบใหม่"
    ]

print(f"\n+ ขั้นตอนถัดไป:")
for i, step in enumerate(next_steps, 1):
    print(f"  {i}. {step}")

print("\n+ สรุป Backend API:")
print("  - รองรับคำภาษามือไทยหลายรูปแบบ")
print("  - มีระบบสำรอง (Fallback) เมื่อ API ไม่พร้อม")  
print("  - ส่งคืนผลลัพธ์ในรูปแบบ JSON")
print("  - เหมาะสำหรับ Integration กับ Frontend")

print("\n+ การใช้งานใน Frontend:")
print("  - POST /generate_sentence")
print("  - ส่งข้อมูล: {\"words\": [\"ผม\", \"รัก\", \"คุณ\"]}")
print("  - ได้รับ: {\"sentences\": [...], \"provider\": \"...\", \"success\": true}")

print("="*50)
print("+ Backend API Demo เสร็จสิ้น!")
print("+ ระบบพร้อมใช้งานแล้ว")
print("="*50)